In [3]:
%env TRANSPORT_CLIENT_ID=
%env TRANSPORT_CLIENT_SECRET=
%env TRANSPORT_ENDPOINT=http://127.0.0.1:5000

env: TRANSPORT_CLIENT_ID=VASnR40OkB79la1t
env: TRANSPORT_CLIENT_SECRET=vD3awmHDe7bVd77VQRm4PL20g7RgEbMF
env: TRANSPORT_ENDPOINT=http://127.0.0.1:5000


In [4]:
from data_uploader.model_interface import DicomProcessor
import pydicom
import base64
import json
import ntpath
from pathlib import Path
from os import walk, path

In [5]:
search_directory = "/mnt/c/source/data/jp2validationdata/VALDAT-017/"

files_to_open = []
for root, _, files in walk(search_directory, topdown=True, onerror=None):
    files_to_open.extend(
        path.join(root, filename)
        for filename in files
        if filename.endswith(".dcm")
    )

for dcm_file in files_to_open:
    dp = DicomProcessor([pydicom.read_file(dcm_file)])
    payload = dp.vision_request()
    image = payload['images'][0]
    j2c_data = base64.b64decode(image['data'])
    with open(f'{Path(dcm_file).stem}.j2c', 'wb') as f:
        f.write(j2c_data)
    del image['data']
    json.dump(payload, open(f'{Path(dcm_file).stem}.json', 'w'))